In [1]:
import sqlalchemy
from src.utils.utils import easyConnect
engine = easyConnect()
conn = engine.connect()

In [32]:
query = sqlalchemy.text('show tables')
result = conn.execute(query)
print(result)
for row in result:
    print(row)

('index_basic_info',)
('stock_basic_info',)


In [43]:
# create a table
meta_data = sqlalchemy.MetaData()
table_example = sqlalchemy.Table(
    "person", meta_data,
    sqlalchemy.Column("id", sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column("name", sqlalchemy.String(128)),
    sqlalchemy.Column("birthday", sqlalchemy.Date, nullable=False)
)
meta_data.create_all(engine)

In [46]:
# INSERT a record
person_insert = table_example.insert()
print(person_insert)
insert_tom = person_insert.values(name='Tom', birthday='20001010')
# another way:
# insert_tom = table_example.insert().values(name='Tom',birthday='20001010')
result = conn.execute(insert_tom)
result = conn.execute(insert_tom) # let's do it 2 times
print(result.inserted_primary_key)
conn.commit()


INSERT INTO person (id, name, birthday) VALUES (:id, :name, :birthday)
(2,)


In [47]:
# insert multiple records
conn.execute(person_insert,[
    {'name': 'alice', 'birthday': '20011010'},
    {'name': 'bob', 'birthday': '20010101'},
    {'name': 'carol', 'birthday': '20020101'},
])
conn.commit()

In [53]:
# SELECT - find data
query = table_example.select()
result_set = conn.execute(query) # returns an ITERATOR, which can be used only once.
for row in result_set:
    print(row)
result_set = conn.execute(query)
for row in result_set:
    print(f"{row[0]}, {row.name}")
result_set = conn.execute(query)
result = result_set.fetchall()
result

(1, 'Tom', datetime.date(2000, 10, 10))
(2, 'Tom', datetime.date(2000, 10, 10))
(3, 'alice', datetime.date(2001, 10, 10))
(4, 'bob', datetime.date(2001, 1, 1))
(5, 'carol', datetime.date(2002, 1, 1))
1, Tom
2, Tom
3, alice
4, bob
5, carol


[(1, 'Tom', datetime.date(2000, 10, 10)),
 (2, 'Tom', datetime.date(2000, 10, 10)),
 (3, 'alice', datetime.date(2001, 10, 10)),
 (4, 'bob', datetime.date(2001, 1, 1)),
 (5, 'carol', datetime.date(2002, 1, 1))]

In [56]:
query = table_example.select().where(table_example.c.birthday > '2001-01-01')
result_set_2 = conn.execute(query)
for row in result_set_2:
    print(row)

(3, 'alice', datetime.date(2001, 10, 10))
(5, 'carol', datetime.date(2002, 1, 1))


In [57]:
from sqlalchemy.sql import and_, or_
query = table_example.select().where(
    or_(table_example.c.birthday > '2001-01-01', # all of the datas match
        and_(
            table_example.c.id >= 1,
            table_example.c.id <= 4
        )
    )
)
result_set_3 = conn.execute(query)
for row in result_set_3:
    print(row)

(1, 'Tom', datetime.date(2000, 10, 10))
(2, 'Tom', datetime.date(2000, 10, 10))
(3, 'alice', datetime.date(2001, 10, 10))
(4, 'bob', datetime.date(2001, 1, 1))
(5, 'carol', datetime.date(2002, 1, 1))


In [58]:
# UPDATE
query = sqlalchemy.text('alter table person add column address varchar(128)')
result_set_4 = conn.execute(query)
result_set_4

In [73]:
reflected_meta_data = sqlalchemy.MetaData()
reflected_person_table = sqlalchemy.Table(
    "person", reflected_meta_data,
    autoload_with=engine
)
update_query = reflected_person_table.update().values(address='aaa')
result = conn.execute(update_query)
conn.commit()
update_query_2 = reflected_person_table.update().values(address='bbb').where(
    reflected_person_table.c.id==1
)
conn.execute(update_query_2)
conn.commit()

In [75]:
# DELETE
insert_query = reflected_person_table.insert().values(name='dave', birthday='2003-01-01',
                                                      address='ddd')
result = conn.execute(insert_query)
conn.commit()
print(result.inserted_primary_key[0])

7


In [76]:
delete_query = reflected_person_table.delete().where(
    reflected_person_table.c.address == 'ddd'
)
conn.execute(delete_query)
conn.commit()


In [2]:
type(engine)

sqlalchemy.engine.base.Engine

In [2]:
import tushare as ts
import pandas as pd
from dotenv import load_dotenv
import os
from utils.utils import *
engine=easyConnect()
df = pd.read_sql('select adj_factor from  {table_name}'.format(table_name=
                                                           'stock_daily_electronic_information'), engine)

In [6]:
df.isnull().sum()

adj_factor    0
dtype: int64

In [6]:
import pandas as pd
from utils.utils import easyConnect
engine = easyConnect()
basic_info_query = "select ts_code from stock_basic_info where list_status='L'"

ts_codes = pd.read_sql(basic_info_query, con=engine)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'demo2.stock_basic_info' doesn't exist")
[SQL: select ts_code from stock_basic_info where list_status='L']
(Background on this error at: https://sqlalche.me/e/20/f405)